In [1]:
from pathlib import Path
import pandas as pd
import sys

ROOT = Path().resolve().parent
if str(ROOT / "src") not in sys.path:
    sys.path.insert(0, str(ROOT / "src"))

from data_workflow.viz import bar_sorted, time_line, histogram_chart, save_fig
from data_workflow.utils import bootstrap_diff_means

DATA = ROOT / "data/processed/analytics_table.parquet"
FIGS = ROOT / "reports/figures"
FIGS.mkdir(parents=True, exist_ok=True)

print("Setup complete!")

Setup complete!


In [2]:
df = pd.read_parquet(DATA)

print(f"Dataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nColumn names:\n{df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")

Dataset shape: 5 rows × 18 columns

Column names:
['order_id', 'user_id', 'amount', 'quantity', 'created_at', 'status', 'status_clean', 'amount__isna', 'quantity__isna', 'date', 'year', 'month', 'dow', 'hour', 'country', 'signup_date', 'amount_winsor', 'amount__is_outlier']

Data types:
order_id                   string[python]
user_id                    string[python]
amount                            Float64
quantity                            Int64
created_at            datetime64[ns, UTC]
status                             object
status_clean                       object
amount__isna                         bool
quantity__isna                       bool
date                               object
year                              float64
month                             float64
dow                                object
hour                              float64
country                            object
signup_date                        object
amount_winsor                     Float6

In [3]:
df.head()

,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor,amount__is_outlier
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,12.0,Monday,10.0,SA,2025-11-15,12.5,False
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,12.0,Monday,11.0,SA,2025-11-20,8.135,False
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,12.0,Tuesday,9.0,AE,2025-11-22,<NA>,<NA>
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,12.0,Wednesday,14.0,SA,2025-11-15,25.0,False
4,A0005,0004,100.0,1,NaT,paid,paid,False,False,None,NaN,NaN,None,NaN,SA,2025-11-25,97.75,True


In [4]:
rev = df.groupby("country", dropna=False)["amount"].sum().reset_index()
rev = rev.sort_values("amount", ascending=False)

print("Revenue by Country:")
print(rev.to_string(index=False))

fig = bar_sorted(rev, "country", "amount", "Total Revenue by Country")
save_fig(fig, FIGS / "revenue_by_country.png")
fig

Revenue by Country:
country  amount
     SA   145.5
     AE     0.0


In [5]:
trend = df.groupby("month", dropna=False)["amount"].sum().reset_index()
trend = trend.sort_values("month")

print("Monthly Revenue Trend:")
print(trend.to_string(index=False))

fig = time_line(trend, "month", "amount", title="Revenue Trend Over Time (Monthly)")
save_fig(fig, FIGS / "revenue_trend_monthly.png")
fig

Monthly Revenue Trend:
 month  amount
  12.0    45.5
   NaN   100.0


In [6]:
print("Order Amount Statistics:")
print(df["amount_winsor"].describe())

fig = histogram_chart(df, "amount_winsor", nbins=40, title="Distribution of Order Amounts (Winsorized)")
save_fig(fig, FIGS / "amount_distribution.png")
fig

Order Amount Statistics:
count          4.0
mean      35.84625
std      41.883474
min          8.135
25%       11.40875
50%          18.75
75%        43.1875
max          97.75
Name: amount_winsor, dtype: Float64


In [7]:
status_counts = df["status_clean"].value_counts().reset_index()
status_counts.columns = ["status", "count"]

print("Order Status Distribution:")
print(status_counts.to_string(index=False))

fig = bar_sorted(status_counts, "status", "count", "Order Status Distribution")
save_fig(fig, FIGS / "status_distribution.png")
fig

Order Status Distribution:
status  count
  paid      4
refund      1


In [8]:
country_orders = df["country"].value_counts().reset_index()
country_orders.columns = ["country", "order_count"]
top10 = country_orders.head(10)

print("Top 10 Countries by Order Count:")
print(top10.to_string(index=False))

fig = bar_sorted(top10, "country", "order_count", "Top 10 Countries by Order Count")
save_fig(fig, FIGS / "top10_countries.png")
fig

Top 10 Countries by Order Count:
country  order_count
     SA            4
     AE            1


In [9]:
import os

saved_figures = [f for f in os.listdir(FIGS) if f.endswith('.png')]
print(f"Exported {len(saved_figures)} figures to {FIGS}:")
for fig in sorted(saved_figures):
    print(f"  {fig}")

Exported 5 figures to /Users/shahad/W2/ETL-pipline/reports/figures:
  amount_distribution.png
  revenue_by_country.png
  revenue_trend_monthly.png
  status_distribution.png
  top10_countries.png
